In [0]:
# dataloader file path
import sys

import time
import torch
import torch.nn as nn
import DataLoader as dataloader
from torch.autograd import Variable
from torchvision import models, transforms
from torch.utils.data import DataLoader as tdataloader

import pandas as pd
import numpy as np
import copy
from google.colab import drive
from matplotlib import pyplot as plt


In [0]:

class  Controller: 
    def __init__(self):
        # track loss and accuracies
        self.num_epochs = 30
        self.batch_size = 32
        self.batch_size_train = 64
        self.class_labels = 196  # Subject to change
        self.loader_works = 1  # multi-process data loading,#loader worker processes.

        self.images_dir = '/content/drive/My Drive/stanford_split/train_all/'
        self.images_dir_test = '/content/drive/My Drive/stanford_split/test_all/'
        self.store_dir = '/content/drive/My Drive/stanford_split/'
        # images_dir='/content/drive/My Drive/25/train/'
        # images_dir_test='/content/drive/My Drive/25/test/'
        self.check_point_path = self.store_dir+'results/'
        self.tr_loss_track, self.val_loss_track, self.tr_acc_track, self.val_acc_track, self.val_acc_history = [], [], [], [], []
        self.USE_GPU = True
        self.datasets = {}
        self.dataloaders = {}
        self.learning_rate = 0.01
        self.optim_type = "sgd"
        self.model_type = "googlenet"

    def setDevice(self):
        if self.USE_GPU and torch.cuda.is_available():
                self.device = torch.device('cuda')
        else:
                self.device = torch.device('cpu')
        print('device running on:', self.device)

    


    def checkpoint(self,model, best_loss, epoch, learning_rate):
        state = {'model': model, 'best_loss': best_loss, 'epoch': epoch, 'rng_state': torch.get_rng_state(),
             'LR': learning_rate}
        torch.save(state, self.check_point_path+str(epoch)+"model.pth")


    def loadmodel(self,model_name):
        # load best model weights to return
        checkpoint_best = torch.load(self.check_point_path+model_name)
        model = checkpoint_best['model']
        return model


    

    def initialize_data(self):
        mean = [0.5, 0.5, 0.5]
        std = [0.5, 0.5, 0.5]

        transform = {'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)]),
        'val': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)]),
        'test': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)])}
        # Loading the dataset with train and val as keys
        # Data divided in training and val using separate script
        datasets = {}
        datasets['train'] = dataloader.ImageDataSet(self.images_dir, fold='train', transformation=transform['train'])
        datasets['val'] = dataloader.ImageDataSet(self.images_dir, fold='val', transformation=transform['val'])
        datasets['test'] = dataloader.ImageDataSet(self.images_dir_test, fold='test', transformation=transform['test'])
        print('datasets', len(datasets['train']))
        print('datasets', len(datasets['val']))

        # Dataloader.util
        self.dataloaders['train'] = tdataloader(datasets['train'], batch_size=self.batch_size_train, shuffle=True, num_workers=self.loader_works)
        self.dataloaders['val'] = tdataloader(datasets['val'], batch_size=self.batch_size, shuffle=True, num_workers=self.loader_works)
        print('dataloaders train ', len(self.dataloaders['train']))
        print('dataloaders valida', len(self.dataloaders['val']))



        self.dataloaders['test'] = tdataloader(datasets['test'], batch_size=self.batch_size, shuffle=True, num_workers=self.loader_works)
        #print('dataloaders', len(dataloaders))


    def training(self):
        # Defining pretrained model
        # model = models.resnet50(pretrained=True)
        if self.model_type=="vgg19":
            model = models.vgg19(pretrained=True)
            model.classifier[6] = nn.Linear(4096,self.class_labels)
        elif self.model_type=="resnet50":
            model = models.resnet50(pretrained=True)
            num_features = model.fc.in_features
            model.fc = nn.Linear(num_features, self.class_labels)
        elif self.model_type =="googlenet":
            model = models.googlenet(pretrained=True)

        model = model.to(self.device)

        criterian = nn.CrossEntropyLoss()

        if self.optim_type=="adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        elif self.optim_type=="sgd":
            optimizer = torch.optim.SGD(model.parameters(), lr=self.learning_rate, momentum=0.9)

        # training
        start_time = time.time()
        best_loss = float('inf')
        best_model = None
        best_acc = 0.0
        best_model_wts = copy.deepcopy(model.state_dict())

        for epoch in range(1, self.num_epochs + 1):
        #print('epoch {}/{}'.format(epoch, num_epochs))
            for t in ['train', 'val']:
                    num_correct = 0.0
                    num_samples = 0
                    r_loss = 0.0
                    running_corrects = 0

                    if t == 'train':
                        # training mode
                        model.train()
                    else:
                        # evaluate model
                        model.eval()

                    count = 0
                    for data in self.dataloaders[t]:
                        count += 1
                        # data has three types files, labels and filename
                        files, labels, filename = data

                        files = Variable(files.to(self.device))  # to gpu or cpu
                        labels = Variable(labels.to(self.device))

                        optimizer.zero_grad()  # clearning old gradient from last step

                        with torch.set_grad_enabled(t == 'train'):
                            pred = model(files)
                            # loss computation
                            loss = criterian(pred, labels)

                            _, prediction = torch.max(pred, 1)

                            # backprop gradients at training time
                            if t == 'train':
                                loss.backward()
                                optimizer.step()

                            # print(t +' iteration {}:loss {}  '.format(count,r_loss))
                        # statistics
                        r_loss += loss.item() * files.size(0)
                        print(t + ' iteration {}:loss {}  '.format(count, r_loss))
                        running_corrects += torch.sum(prediction == labels.data)
                    epoch_loss = r_loss / len(self.dataloaders[t].dataset)
                    epoch_acc = running_corrects.double() / len(self.dataloaders[t].dataset)

                    print('{} Loss: {:.4f} Acc: {:.4f}'.format(t, epoch_loss, epoch_acc))

                    # print(t +' epoch {}:loss {}  '.format(epoch,r_loss))

                    # deep copy the model
                    if t == 'val' and epoch_acc > best_acc:
                        best_acc = epoch_acc
                        best_model_wts = copy.deepcopy(model.state_dict())
                        self.checkpoint(best_model_wts, best_loss, epoch, self.learning_rate)
                    if t == 'val':
                        self.val_acc_history.append(epoch_acc.item())
                        self.val_loss_track.append(epoch_loss)

                    if t == 'train':
                        self.tr_loss_track.append(epoch_loss)
                        self.tr_acc_track.append(epoch_acc.item())



        time_elapsed = time.time() - start_time
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best val Acc: {:4f}'.format(best_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
        # updating best model in checkpoint

        return model







    def predict(self,model):
        torch.cuda.empty_cache()

        num_correct = 0
        num_samples = 0
        count = 0
        for data in self.dataloaders['test']:
            count += 1
            files, labels, filename = data
            # load the data to GPU / CPU
            image_data = Variable(files.to(self.device))
            labels = Variable(labels.to(self.device))
            output = model(image_data)
            # print('output', output)
            _, prediction = torch.max(output.data, 1)
            # print('filename',filename,'\nlabels',labels,'\nprediction',prediction)
            num_correct += torch.sum(prediction == labels)
            num_samples += prediction.size(0)
            print('Iteration', count)
        acc = float(num_correct) / num_samples
        print('Accuracy of test data :', acc)


    def plot_losses_both(loss_track1,loss_track2,track1_label="Training Loss",track2_label="Validation loss",plt_label="Loss"):
        ax = plt.subplot(111)
        plt.plot(loss_track1, c='b', label=track1_label)
        plt.plot(loss_track2, c='g', label=track2_label)
        plt.title(plt_label)
        ax.legend()
        plt.savefig(store_dir+"train_val.png")

    def plot_loss_Accu(val_acc,xlab="Num epochs",ylab="Validation accuracy"):
        plt.plot(val_acc, c='b')
        plt.xlabel(xlab)
        plt.ylabel(ylab)
        plt.savefig(store_dir+"val_acc.png")


In [33]:

sys.path.insert(1, '/content/drive/My Drive/Colab Notebooks')
drive.mount('/content/drive')


cont = Controller()
cont.setDevice()
cont.initialize_data()



    

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
device running on: cuda
################# train #############
#######################################
################# val #############
#######################################
################# test #############
#######################################
datasets 6380
datasets 1764
dataloaders train  100
dataloaders valida 56


In [22]:
print('Training began....')
best_model = cont.training()


Training began....
train iteration 1:loss 502.53350830078125  
train iteration 2:loss 1011.4563903808594  
train iteration 3:loss 1498.6766967773438  
train iteration 4:loss 1993.1148986816406  
train iteration 5:loss 2478.762481689453  
train iteration 6:loss 2959.5226440429688  
train iteration 7:loss 3441.8888549804688  
train iteration 8:loss 3914.212615966797  
train iteration 9:loss 4373.225189208984  
train iteration 10:loss 4828.6495361328125  
train iteration 11:loss 5268.524658203125  
train iteration 12:loss 5709.688507080078  
train iteration 13:loss 6144.218292236328  
train iteration 14:loss 6567.330017089844  
train iteration 15:loss 6987.752349853516  
train iteration 16:loss 7400.547088623047  
train iteration 17:loss 7820.063262939453  
train iteration 18:loss 8221.10952758789  
train iteration 19:loss 8621.29931640625  
train iteration 20:loss 9022.073303222656  
train iteration 21:loss 9425.836822509766  
train iteration 22:loss 9832.518493652344  
train iteration 2

In [44]:
print('Testing began....')
best_model = cont.loadmodel("19model.pth")
google_net = models.googlenet(pretrained=True)
google_net.to(cont.device)
#print(best_model)
google_net.load_state_dict(best_model)
cont.predict(google_net)


Testing began....
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Ite